In [39]:
import os as os
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict



In [2]:
"""combine the files from the lowest directory into one file (NGO/IGO/Press)"""
def combine_files_by_organisation(output_dir, root_path):
    os.makedirs(output_dir, exist_ok=True) 
    for folder_org in sorted(os.listdir(root_path)):  
        org_path = os.path.join(root_path, folder_org)
        combined_df = []
        output_file = os.path.join(output_dir, f"{folder_org}.csv")

        #Combine files from each subdirectory
        for sub_dir in sorted(os.listdir(org_path)):
            sub_path = os.path.join(org_path, sub_dir)
            if os.path.isdir(sub_path):
                for filename in sorted(os.listdir(sub_path)):
                    file_path = os.path.join(sub_path, filename)
                    if os.path.isfile(file_path) and file_path.endswith(".csv"):
                        df = pd.read_csv(file_path)
                        combined_df.append(df)
        if combined_df:
            final_df = pd.concat(combined_df, ignore_index=True)
            final_df.to_csv(output_file, index=False)
            print(f"CSV saved to: {output_file}")
        else:
            print(f"No CSV files found.")

In [9]:
input_path = "new_cluster/original_labels/by_org"  
output_path = "new_cluster/custom_files/labels"  

combine_files_by_organisation(output_path, input_path)

input_path = "new_cluster/original_dimensions/by_org"  
output_path = "new_cluster/custom_files/dimensions"  

combine_files_by_organisation(output_path, input_path)


CSV saved to: new_cluster/custom/labels\IGO.csv
CSV saved to: new_cluster/custom/labels\NGO.csv
CSV saved to: new_cluster/custom/labels\Presse.csv


In [37]:
def plot_label_means(file_paths, custom_names, output_dir, name="Temp", colors=None):
    if not file_paths or len(file_paths) != len(custom_names):
        print("Error: Ensure file_paths and custom_names are non-empty and have the same length.")
        return

    df_list = []
    for file_path, label in zip(file_paths, custom_names):
        df = pd.read_csv(file_path)
        df_mean = df.mean().to_frame(name=label)
        df_mean.index.name = 'Label'
        df_list.append(df_mean)
    merged_df = pd.concat(df_list, axis=1)
    merged_df = merged_df.iloc[::-1]
    plt.figure(figsize=(8, 6))
    merged_df.plot(kind='barh', figsize=(8, 6), width=0.7, color=colors)
    
    plt.xlabel('')
    plt.ylabel('')
    plt.title(name)
    plt.legend(custom_names,loc='upper right', bbox_to_anchor=(1, 1),prop={'size': 9.5})
    plt.tight_layout()
    plt.xlim(0, 1)
    for x in [0.2, 0.4, 0.6, 0.8]:
        plt.axvline(x=x, color='grey', linestyle='--')
    output_file = os.path.join(output_dir, f"{name}.png")
    plt.savefig(output_file, bbox_inches="tight")
    plt.close()
    print(f"Plot saved: {output_file}")


In [38]:
plot_name = "NGOs vs IGOs vs Newspapers"

file_paths = [
    "new_cluster/custom_files/original_labels/NGO.csv",
    "new_cluster/custom_files/original_labels/IGO.csv",
    "new_cluster/custom_files/original_labels/Presse.csv"
]

custom_names = ["NGOs", "IGOs", "Newspapers"]
output_dir = "new_cluster/plots/"

#colors (optional)
custom_colors = ["#1F77B4", "#FF7F0E", "#2CA02C"]

# Generate plot
plot_label_means(file_paths, custom_names, output_dir, plot_name, colors=custom_colors)

Plot saved: new_cluster/plots/NGOs vs IGOs vs Newspapers.png


<Figure size 800x600 with 0 Axes>

In [49]:
plot_name = "Left vs Right - Newspapers"

file_paths = [
    "new_cluster/custom_files/labels/center.csv",
    "new_cluster/custom_files/labels/center-left.csv",
    "new_cluster/custom_files/labels/right.csv"
]

custom_names = ["center", "center-left", "right"]
output_dir = "new_cluster/plots/"

#colors (optional)
custom_colors = ["#FFA500", "#FF0000", "#0000FF"]

# Generate plot
plot_label_means(file_paths, custom_names, output_dir, plot_name, colors=custom_colors)

Plot saved: new_cluster/plots/Left vs Right - Newspapers.png


<Figure size 800x600 with 0 Axes>

In [48]:

def combine_csv_by_organization(root_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for category in os.listdir(root_path):
        category_path = os.path.join(root_path, category)
        if not os.path.isdir(category_path):
            continue 
        org_files = defaultdict(list)
        for subdir, _, files in os.walk(category_path):
            for file in files:
                if file.endswith(".csv"):
                    prefix = file.split('-')[0] 
                    org_files[prefix].append(os.path.join(subdir, file))
        category_output_path = os.path.join(output_dir, category)
        os.makedirs(category_output_path, exist_ok=True)

        for org, file_list in org_files.items():
            combined_data = pd.concat([pd.read_csv(f) for f in file_list], ignore_index=True)
            output_file = os.path.join(category_output_path, f"{org}.csv")
            combined_data.to_csv(output_file, index=False)
            print(f"Saved: {output_file}")

combine_csv_by_organization("new_cluster/original_dimensions/by_org", "new_cluster/custom_files/dimensions")

Saved: new_cluster/test\IGO\IPCC.csv
Saved: new_cluster/test\IGO\REDD.csv
Saved: new_cluster/test\IGO\UNDP.csv
Saved: new_cluster/test\IGO\UNEP.csv
Saved: new_cluster/test\IGO\WB.csv
Saved: new_cluster/test\IGO\WMO.csv
Saved: new_cluster/test\IGO\WHO.csv
Saved: new_cluster/test\IGO\UNCDF.csv
Saved: new_cluster/test\IGO\UNFCCC.csv
Saved: new_cluster/test\NGO\CG.csv
Saved: new_cluster/test\NGO\EDF.csv
Saved: new_cluster/test\NGO\FoEI.csv
Saved: new_cluster/test\NGO\GP.csv
Saved: new_cluster/test\NGO\NRDC.csv
Saved: new_cluster/test\NGO\OX.csv
Saved: new_cluster/test\NGO\WWF.csv
Saved: new_cluster/test\NGO\Action.csv
Saved: new_cluster/test\NGO\CAN.csv
Saved: new_cluster/test\NGO\CARE.csv
Saved: new_cluster/test\NGO\CI.csv
Saved: new_cluster/test\NGO\CIDSE.csv
Saved: new_cluster/test\NGO\EJF.csv
Saved: new_cluster/test\NGO\GGJA.csv
Saved: new_cluster/test\NGO\IBON.csv
Saved: new_cluster/test\NGO\IICAT.csv
Saved: new_cluster/test\NGO\OCI.csv
Saved: new_cluster/test\NGO\OXFAM.csv
Saved: new